In [1]:
import os
import pandas as pd 
import cv2  
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
from sklearn.manifold import TSNE

In [2]:
train_path = '../data/samples10-m-2'
#train_path = '../data/train'
cameras = os.listdir(train_path)

train_images = []
for camera in cameras:
    for fname in sorted(os.listdir(train_path+ '/'+ camera)):
        train_images.append((camera, fname))

train = pd.DataFrame(train_images, columns=['camera', 'fname'])
print(train.shape)
train.sample(5)

(180, 2)


,camera,fname
104,iPhone-4s,(iP4s)10-r0.5-512.jpg
132,iPhone-4s,(iP4s)4-r0.8-512.jpg
165,iPhone-4s,(iP4s)8-j70-512.jpg
73,HTC-1-M7,(HTC-1-M7)8-g0.8-512.jpg
49,HTC-1-M7,(HTC-1-M7)5-j90-512.jpg


In [11]:
data = pd.DataFrame(columns=['image','camera'])
images = []
cameras = []
input_path = '../data/samples10-m-2/'
#input_path = '../data/train/'

for img in train_images:
    im_path = input_path+img[0]+'/'+img[1]
    image = cv2.imread(im_path)  
    h, w, c = image.shape
    x1 = (w//2 - 64)
    y1 = (w//2 + 64)
    x2 = (h//2 - 64)
    y2 = (h//2 + 64)
    cropped_image = image[x2:y2, x1:y1]
    
    #ni = image.reshape(-1)
    #images.append (ni)
    #images.append(image)
    images.append(cropped_image)
    cameras.append(img[0])

X= np.array(images)
print('Images shape:', X.shape)

Images shape: (180, 128, 128, 3)


In [12]:
#y = np.array(cameras)
#y.shape

labels, levels = pd.factorize(cameras)
y = labels

In [13]:
from sklearn.model_selection import train_test_split
X_train_, X_test, y_train_, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_, y_train_, test_size=0.33, random_state=42)

In [14]:
np.unique(y)

array([0, 1], dtype=int64)

In [15]:
label_map = {
  0: 'HTC-1-M7',
  1: 'iPhone-4s'
}

In [16]:
from keras.callbacks import EarlyStopping
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D
from keras.optimizers import Adam
from keras.layers.pooling import MaxPooling2D
from keras.utils import to_categorical

# For reproducibility
np.random.seed(1000)

# Create the model
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(lr=0.0001, decay=1e-6),
                  metrics=['accuracy'])


In [19]:

# Train the model
model.fit(X_train / 255.0, to_categorical(y_train,2),
              batch_size=32,
              shuffle=True,
              epochs=50,
              validation_data=(X_valid / 255.0, to_categorical(y_valid)),
              callbacks=[EarlyStopping(min_delta=0.001, patience=3)])


Train on 80 samples, validate on 40 samples
Epoch 1/50
80/80 [==============================] - 11s 136ms/step - loss: 0.4216 - acc: 0.8875 - val_loss: 0.5325 - val_acc: 0.8250
Epoch 2/50
80/80 [==============================] - 5s 61ms/step - loss: 0.4475 - acc: 0.8375 - val_loss: 0.5421 - val_acc: 0.8000
Epoch 3/50
80/80 [==============================] - 5s 62ms/step - loss: 0.4726 - acc: 0.8000 - val_loss: 0.5350 - val_acc: 0.8000
Epoch 4/50
80/80 [==============================] - 5s 59ms/step - loss: 0.4451 - acc: 0.7875 - val_loss: 0.5130 - val_acc: 0.8250
Epoch 5/50
80/80 [==============================] - 5s 60ms/step - loss: 0.4178 - acc: 0.8500 - val_loss: 0.4752 - val_acc: 0.8500
Epoch 6/50
80/80 [==============================] - 5s 58ms/step - loss: 0.3582 - acc: 0.9125 - val_loss: 0.4260 - val_acc: 0.9250
Epoch 7/50
80/80 [==============================] - 5s 59ms/step - loss: 0.3206 - acc: 0.9500 - val_loss: 0.3729 - val_acc: 0.9750
Epoch 8/50
80/80 [===================

In [30]:

# Evaluate the model
scores = model.evaluate(X_valid / 255.0, to_categorical(y_valid,2))

print('Loss: %.3f' % scores[0])
print('Accuracy: %.3f' % scores[1])

40/40 [==============================] - 1s 25ms/step
Loss: 0.023
Accuracy: 1.000


In [36]:
from sklearn.metrics import confusion_matrix

predicted = model.predict(X_test)


In [39]:
predicted

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,

In [41]:
to_categorical(y_test,2)

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,

In [40]:
y_test

array([0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0], dtype=int64)

In [ ]:
matrix = confusion_matrix(to_categorical(y_test,2), predicted)
print(matrix)

In [22]:
from keras.wrappers.scikit_learn import KerasClassifier

estimator = KerasClassifier(build_fn=model, epochs=200, batch_size=4, verbose=0)

In [26]:
from sklearn.model_selection import KFold
import numpy

seed = 7
numpy.random.seed(seed)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

In [29]:
from sklearn.model_selection import cross_val_score
results = cross_val_score(estimator, X_train / 255.0, to_categorical(y_train,2), cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

TypeError: can't pickle _thread.lock objects